In [53]:
import mysql.connector
from mysql.connector import Error
import json, csv


In [2]:
# Connect to MySQL

try:
    mydb = mysql.connector.connect(host='localhost', user='root', 
                        auth_plugin='mysql_native_password',
                        password='=~n4+5KVeNL-8c#')#give ur username, password
except Error as e:
    print("Error while connecting to MySQL", e)
        

In [3]:
#create new mysql database and use it
if mydb.is_connected():
        # uncomment this line if running for the first time
        # try:
                # cursor.execute("CREATE DATABASE VideoGameSales") 
                # print("Database is created")
        # except Error as e:
                # print("Error creating database: ", e)
        cursor = mydb.cursor()
        cursor.execute("USE videogamesales")

In [6]:
#import data from relational database
try:
    cursor.execute("DROP TABLE IF EXISTS rdbdata")
    cursor.execute("CREATE TABLE rdbdata SELECT * FROM `rdb-vgsales`.genres")
except Error as e:
    print("Error fetching data from the relational database: ", e)

In [19]:
# Load the JSON data from the nosql database
with open('gamesmongo.json', 'r') as f:
    data = json.load(f)

# Create a new table with columns based on the JSON data
sql_columns = []
for key in data[0].keys():
    if key =='sales':
        sql_columns.append(f"{key} FLOAT")
    else:
        sql_columns.append(f"{key} VARCHAR(255)")
sql_columns = ', '.join(sql_columns)
print('column names: ', sql_columns)

# Create table to store values
try:
    cursor.execute("DROP TABLE IF EXISTS nosql_data")
    cursor.execute(f"CREATE TABLE nosql_data ({sql_columns})")
except Error as e:
    print("Error creating table for data from the nosql database: ", e)

# insert data into the new table
try:
    for item in data:
        id = str(item['_id']['$oid'])
        release_year = str(item['release_year'])
        genre = str(item['genre'])
        region = str(item['region'])
        sales = float(item['sales'])

        cursor.execute(f"INSERT INTO nosql_data (_id, release_year, genre, region, sales) VALUES ('{id}', '{release_year}', '{genre}', '{region}', {sales})")
except Error as e:
    print("Error importing data from the nosql database: ", e)

# Commit the changes
mydb.commit()


_id VARCHAR(255), release_year VARCHAR(255), genre VARCHAR(255), region VARCHAR(255), sales FLOAT


In [48]:
#Upload data froma csv

with open('vgsales.csv', 'r') as csvfile:
    # Read the CSV file
    csvreader = csv.reader(csvfile)
    # Skip the header row
    header = next(csvreader)
    sql_columns = []
    for column in header:
        if 'sales' in column:
            sql_columns.append(f"{column} FLOAT")
        else:
            sql_columns.append(f"{column} VARCHAR(255)")
    sql_columns = ', '.join(sql_columns)
    header = ", ".join([value for value in header])

    #create table for csv data
    try:
        cursor.execute("DROP TABLE IF EXISTS csvData")
        sql =  f"CREATE TABLE csvData ({sql_columns})"
        print(sql)
        cursor.execute(sql)
    except Error as e:
        print("Error creating table for data from the csv data: ", e)

    # Loop through the rows in the CSV file
    try:
        for row in csvreader:
            # Insert the row into the MySQL database
            row[1] = row[1].replace("'", r"\'")
            row[5] = row[5].replace("'", r"\'")
            strings = "".join([f"'{value}', " for value in row[:6]])
            numbers = ", ".join([f"{float(value)}" for value in row[6:]])
            sql = f"INSERT INTO csvData ({header}) VALUES ({strings}{numbers})"
            cursor.execute(sql)
        mydb.commit()
    except Error as e:
        print(sql)
        print("Error importing data from csv: ", e)

CREATE TABLE csvData (Ranking VARCHAR(255), Game VARCHAR(255), Platform VARCHAR(255), release_year VARCHAR(255), Genre VARCHAR(255), Publisher VARCHAR(255), NA_Sales VARCHAR(255), EU_Sales VARCHAR(255), JP_Sales VARCHAR(255), Other_Sales VARCHAR(255), Global_Sales VARCHAR(255))


In [54]:
# write some queries to test the data mart

# What is the total sales for each genre?
try:
    cursor.execute("SELECT csvData.genre, SUM(NA_Sales + EU_Sales+ JP_Sales+ Other_Sales+ Global_Sales + sales) AS total_sales FROM csvData inner Join nosql_data GROUP BY csvData.genre")
    result = cursor.fetchall()
    print('Total sales for each genre: ', result)
except Error as e:
    print("Error running query: ", e)

Total sales for each genre:  [('Sports', 46558385.918456875), ('Platform', 18865104.531691648), ('Racing', 24878845.848329213), ('Role-Playing', 29855384.80606742), ('Puzzle', 11217049.974546826), ('Misc', 33822456.943713434), ('Shooter', 27144777.88788956), ('Simulation', 16817380.37188074), ('Action', 65300763.20961894), ('Fighting', 16704134.212127607), ('Adventure', 23608668.52769928), ('Strategy', 12690969.833614325)]


In [56]:
# What region had the most sales?
try:
    cursor.execute("SELECT region, SUM(sales) AS total_sales FROM nosql_data GROUP BY region")
    result = cursor.fetchall()
    print('Total sales for each region: ', result)
except Error as e:
    print("Error running query: ", e)

Total sales for each region:  [('North America', 4341.690005583689), ('European Union', 2399.6799919568), ('Japan', 1290.700002539903), ('Other', 782.6499985102564), ('Worldwide', 8818.919992174953)]


In [61]:
# Which genre had the best ratings?
try:
    cursor.execute("SELECT genre, AVG(Critic_Score), AVG(User_Score) FROM rdbdata GROUP BY genre")
    result = cursor.fetchall()
    print('Average Rating for each genre (critic is out of 100, user is out of 10): ', result)
except Error as e:
    print("Error running query: ", e)

Average Rating for each genre (critic is out of 100, user is out of 10):  [('Sports', Decimal('71.9682'), 5.06477572559367), ('Platform', Decimal('68.0584'), 5.511463844797178), ('Racing', Decimal('67.9636'), 5.160411899313506), ('Role-Playing', Decimal('72.6526'), 7.323803363518762), ('Puzzle', Decimal('67.4241'), 2.7091445427728607), ('Misc', Decimal('66.6195'), 3.371283783783784), ('Shooter', Decimal('70.1811'), 6.360410557184758), ('Simulation', Decimal('68.6193'), 4.21701030927835), ('Action', Decimal('66.6291'), 5.851722574796006), ('Fighting', Decimal('69.2176'), 6.652283105022839), ('Adventure', Decimal('65.3313'), 4.662091503267976), ('Strategy', Decimal('72.0861'), 6.482285714285712), (None, None, None)]
